# TCC Pós Graduação Ciência de Dados e Big Data
### Obtenção de dados

In [1]:
import io
import os
from time import time
import requests
import pandas as pd
import numpy as np
import PyPDF2
import re
import csv

DIR_DATASET = r'/Users/dev-rocks/Documents/TCC - Data Science e Big Data/Projeto/datasets/'
URL_ARQUIVO_PROPOSICOES = r'http://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/'
URL_ARQUIVO_TEMAS = r'http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/csv/'

In [54]:
def extrairDatasetsPorAno(anoInicio:int, anoFim:int):
    anosParaExtracao = range(anoInicio, anoFim+1)
    for ano in anosParaExtracao:
        urlArquivoProposicao = f'{URL_ARQUIVO_PROPOSICOES}proposicoes-{ano}.csv'
        urlArquivoTemas = f'{URL_ARQUIVO_TEMAS}proposicoesTemas-{ano}.csv'

        arquivoProposicao = requests.get(urlArquivoProposicao)
        with open(f'{DIR_DATASET}/proposicoes/proposicoes-{ano}.csv','wb') as arquivo:
            arquivo.write(arquivoProposicao.content)
            print(f'[OK    ] Arquivo baixado: {urlArquivoProposicao} ({len(arquivoProposicao.content)} bytes)')

        arquivoTemas = requests.get(urlArquivoTemas)
        with open(f'{DIR_DATASET}/temas/proposicoesTemas-{ano}.csv','wb') as arquivo:
            arquivo.write(arquivoTemas.content)

        print(f'[OK    ] Arquivo baixado: {urlArquivoTemas} ({len(arquivoTemas.content)} bytes)')
    print(f'[OK    ] Finalizada extração dos datasets de preposição e classificação temática de {anoInicio} a {anoFim}')

In [55]:
extrairDatasetsPorAno(2018,2021)

[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-2018.csv (15727867) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/csv/proposicoesTemas-2018.csv (614772) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-2019.csv (31523314) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/csv/proposicoesTemas-2019.csv (1715368) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-2020.csv (24161880) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/csv/proposicoesTemas-2020.csv (2246413) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-2021.csv (33474989) bytes
[OK    ] Arquivo baixado: http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/csv/proposicoesTemas-2021.c

In [71]:
def gerarDataframeProposicoes():
    dfProposicoesList = []

    for arquivo in os.listdir(f'{DIR_DATASET}proposicoes/'):
        try:
            anoDataset = arquivo.split('.')[0][-4:]
            df = pd.read_csv(f'{DIR_DATASET}proposicoes/{arquivo}', sep=';', low_memory=False)
            df.loc[:,'anoDataset'] = anoDataset 
            dfProposicoesList.append(df)
            print(f'[OK    ] [{anoDataset}] {arquivo}')
        except Exception as e:
            print(f'[ERRO  ] {arquivo}')
    
    return  pd.concat(dfProposicoesList)

In [72]:
def gerarDataframeTemas():
    dfTemasList = []
    for arquivo in os.listdir(f'{DIR_DATASET}temas/'):
        try:
            anoDataset = arquivo.split('.')[0][-4:]
            df = pd.read_csv(f'{DIR_DATASET}temas/{arquivo}', sep=';', low_memory=False)
            df.loc[:,'anoDataset'] = anoDataset
            dfTemasList.append(df)
            print(f'[OK    ] [{anoDataset}] {arquivo}')
        except Exception as e:
            print(f'[ERRO  ]: {arquivo}')
            
    return pd.concat(dfTemasList)

In [29]:
def extrairInteiroTeorPdf(conteudo):
    textoPaginas = ''
    pdf = io.BytesIO(conteudo)
    pdfReader = PyPDF2.PdfFileReader(pdf)
    numeroPaginas = pdfReader.numPages

    for p in range(numeroPaginas):
        pagina = pdfReader.getPage(p)
        textoPaginas += pagina.extractText()
  
    return textoPaginas

In [30]:
def extrairInteiroTeorHtml(conteudo):
    TAG_HTML = re.compile(r'<[^>]+>')
    return TAG_HTML.sub('',conteudo)

In [73]:
dfProposicoes = gerarDataframeProposicoes()
dfProposicoes[['id','ementa','anoDataset']]


[OK    ] [2019] proposicoes-2019.csv
[OK    ] [2018] proposicoes-2018.csv
[ERRO  ] .DS_Store
[OK    ] [2020] proposicoes-2020.csv
[OK    ] [2021] proposicoes-2021.csv
[ERRO  ] inteiro-teor


,id,ementa,anoDataset
0,308880,Dá nova redação aos arts. 20 e 123 do Decreto-...,2019
1,317970,"Altera a Lei nº 8.666, de 21 de junho de 1993,...",2019
2,427339,"Altera o art. 125 da Lei nº 9.279, de 14 de ma...",2019
3,501638,Institui o Programa Cidade Amiga do Idoso.,2019
4,512431,Proíbe a utilização de mensagens subliminares ...,2019
...,...,...,...
39759,2316058,Submete à apreciação do Congresso Nacional o t...,2021
39760,2316178,Submete à apreciação do Congresso Nacional o t...,2021
39761,2316179,Submete à apreciação do Congresso Nacional o t...,2021
39762,2316200,Submete à apreciação do Congresso Nacional o t...,2021


In [74]:
dfTemas = gerarDataframeTemas()
dfTemas.info()

[OK    ] [2019] proposicoesTemas-2019.csv
[OK    ] [2018] proposicoesTemas-2018.csv
[ERRO  ]: .DS_Store
[OK    ] [2020] proposicoesTemas-2020.csv
[OK    ] [2021] proposicoesTemas-2021.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 53268 entries, 0 to 14915
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uriProposicao  53268 non-null  object
 1   siglaTipo      53268 non-null  object
 2   numero         53268 non-null  int64 
 3   ano            53268 non-null  int64 
 4   codTema        53268 non-null  int64 
 5   tema           53268 non-null  object
 6   relevancia     53268 non-null  int64 
 7   anoDataset     53268 non-null  object
dtypes: int64(4), object(4)
memory usage: 3.7+ MB


In [17]:
dfInteiroTeor = dfProposicoes.loc[:,['id','urlInteiroTeor']]
dfInteiroTeor.loc[:,'statusExtracao'] = 0
dfInteiroTeor.loc[:,'tipoDocumento'] = np.NaN
dfInteiroTeor.loc[:,'tamanhoEmBytes'] = 0
dfInteiroTeor.loc[:,'inteiroTeor'] = np.NaN

In [18]:
dfInteiroTeor.describe()

,id,statusExtracao,tipoDocumento,tamanhoEmBytes,inteiroTeor
count,1.263980e+05,126398.0,0.0,126398.0,0.0
mean,2.241709e+06,0.0,NaN,0.0,NaN
std,4.665378e+04,0.0,NaN,0.0,NaN
min,3.088800e+05,0.0,NaN,0.0,NaN
25%,2.205283e+06,0.0,NaN,0.0,NaN
50%,2.243230e+06,0.0,NaN,0.0,NaN
75%,2.278615e+06,0.0,NaN,0.0,NaN
max,2.316251e+06,0.0,NaN,0.0,NaN


In [75]:
#Salvar datasets consolidados em um arquivo csv
dfProposicoes.to_csv(f'{DIR_DATASET}proposicoes.csv',index=False,sep=';',quoting=csv.QUOTE_ALL,  escapechar="\\")
print(f'[OK    ] Arquivo salvo: {DIR_DATASET}proposicoes.csv')

dfTemas.to_csv(f'{DIR_DATASET}temas.csv',index=False,sep=';',quoting=csv.QUOTE_ALL,  escapechar="\\")
print(f'[OK    ] Arquivo salvo: {DIR_DATASET}temas.csv')

dfInteiroTeor.to_csv(f'{DIR_DATASET}inteiroTeor.csv',index=False,sep=';',quoting=csv.QUOTE_ALL,  escapechar="\\")
print(f'[OK    ] Arquivo salvo: {DIR_DATASET}inteiroTeor.csv')

[OK    ] Arquivo salvo: /Users/dev-rocks/Documents/TCC - Data Science e Big Data/Projeto/datasets/proposicoes.csv
[OK    ] Arquivo salvo: /Users/dev-rocks/Documents/TCC - Data Science e Big Data/Projeto/datasets/temas.csv


## Extrair inteiro teor das proposições

In [88]:
def extrairInteiroTeor(df, qtdLimite=0):
    dfParaExtracao = df.loc[df['statusExtracao']==0]
    print(f'[INFO  ] Proposicoes para extracao de inteiro teor: {len(dfParaExtracao)}')
    
    sessao = requests.Session()
    contador = 0
    t0 = time()
    for index,linha in dfParaExtracao.iterrows():
        try:
            url = linha['urlInteiroTeor']
            idPreposicao = linha['id']
            indexPreposicao =  dfInteiroTeor.loc[df['id']==idPreposicao].index.item()

            if contador%100==0:
                print(f'[INFO  ] [{indexPreposicao}][{idPreposicao}][{url}]')

            if type(url) == float:
                df.loc[indexPreposicao,['statusExtracao','tipoDocumento']] = [4,'Proposicao s/ url inteiro teor']
                continue

            response = sessao.get(url)
            headers = response.headers
            tipoDocumento = str(headers['content-type'])
            statusExtracao=0
            inteiroTeor =''
            tokensInteiroTeor = ''
            tamanhoConteudo = 0

            if tipoDocumento =='application/pdf':
                inteiroTeor = extrairInteiroTeorPdf(response.content)
                tamanhoConteudo = len(response.content)
                statusExtracao = 1
            elif tipoDocumento=='text/html;charset=utf-8':
                idTeor = url.split('=')[-1]
                response = sessao.get(f'https://www.camara.leg.br/internet/ordemdodia/integras/{idTeor}.htm')
                inteiroTeor = extrairInteiroTeorHtml(response.text)
                tamanhoConteudo = len(response.content)
                statusExtracao = 1
            else:
                print(f'[ALERTA] [{url}]{tipoDocumento}')
                statusExtracao = 2

            
            df.loc[indexPreposicao,
                              ['statusExtracao',
                               'tipoDocumento',
                               'inteiroTeor',
                               'tamanhoEmBytes']] = [statusExtracao,tipoDocumento,inteiroTeor,tamanhoConteudo]
            
            if qtdLimite>0 and contador ==qtdLimite:
                break
            
            contador+=1
        except Exception as e:
            print(f'[ERRO  ] Erro no indice: {index} {e.args}')
            df.loc[indexPreposicao,['statusExtracao','tipoDocumento']] = [3,e.args]


    sessao.close()
    duracao = time()-t0

    print(f'[INFO  ] Extraído teor de {contador} proposicoes em {duracao:.0f} segundos. Media={(contador/duracao):.2f} prop/seg' )
    df.to_csv(f'{DIR_DATASET}/inteiroTeor.csv',index=False,sep=';',quoting=csv.QUOTE_ALL,  escapechar="\\")


In [3]:
dfInteiroTeor = pd.read_csv(f'{DIR_DATASET}inteiroTeor.csv', sep=';', low_memory=False)

count    1.263980e+05
mean     8.902567e+05
std      9.284161e+06
min      0.000000e+00
25%      1.092812e+05
50%      1.453005e+05
75%      2.518335e+05
max      8.082264e+08
Name: tamanhoEmBytes, dtype: float64

In [5]:
dfInteiroTeor['tamanhoEmBytes'].describe().round(0)

count       126398.0
mean        890257.0
std        9284161.0
min              0.0
25%         109281.0
50%         145300.0
75%         251834.0
max      808226386.0
Name: tamanhoEmBytes, dtype: float64

In [25]:
dfSemUrlInteiroTeor = dfInteiroTeor.index[dfInteiroTeor['urlInteiroTeor'].isnull()]
print(f'[INFO    ] Total de proposições sem inteiro teor: {len(dfSemUrlInteiroTeor)}')

dfInteiroTeor.loc[dfInteiroTeor['urlInteiroTeor'].isnull(),
                 ['statusExtracao','tipoDocumento']] = [4,'Proposicao s/ url inteiro teor']

dfInteiroTeor.loc[dfInteiroTeor['statusExtracao']==4]


[INFO    ] Total de proposições sem inteiro teor: 636


,id,urlInteiroTeor,statusExtracao,tipoDocumento,tamanhoEmBytes,inteiroTeor
27,2190174,NaN,4,Proposicao s/ url inteiro teor,0,NaN
2270,2193079,NaN,4,Proposicao s/ url inteiro teor,0,NaN
4197,2195361,NaN,4,Proposicao s/ url inteiro teor,0,NaN
5657,2197072,NaN,4,Proposicao s/ url inteiro teor,0,NaN
5658,2197073,NaN,4,Proposicao s/ url inteiro teor,0,NaN
...,...,...,...,...,...,...
124943,2312236,NaN,4,Proposicao s/ url inteiro teor,0,NaN
125184,2312500,NaN,4,Proposicao s/ url inteiro teor,0,NaN
125185,2312501,NaN,4,Proposicao s/ url inteiro teor,0,NaN
125563,2312911,NaN,4,Proposicao s/ url inteiro teor,0,NaN


In [113]:
extrairInteiroTeor(dfInteiroTeor,3000)

[INFO  ] Proposicoes para extracao de inteiro teor: 0
[INFO  ] Extraído teor de 0 proposicoes em 0 segundos. Media=0.00 prop/seg


In [96]:
dfInteiroTeor = pd.read_csv(f'{DIR_DATASET}inteiroTeor.csv', sep=';', low_memory=False)


In [123]:
dfInteiroTeor.loc[(dfInteiroTeor['statusExtracao']==1)&(dfInteiroTeor['inteiroTeor'].isnull())]


,id,urlInteiroTeor,statusExtracao,tipoDocumento,tamanhoEmBytes,inteiroTeor
18,2080812,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,53091,NaN
19,2111588,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,968013,NaN
326,2190692,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,572179,NaN
2304,2193133,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,25333,NaN
2373,2193216,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,695994,NaN
...,...,...,...,...,...,...
126393,2316058,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,143370,NaN
126394,2316178,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,319115,NaN
126395,2316179,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,154366,NaN
126396,2316200,http://www.camara.gov.br/proposicoesWeb/prop_m...,1,application/pdf,175666,NaN


In [124]:
dfInteiroTeor.loc[(dfInteiroTeor['statusExtracao']==1)&(dfInteiroTeor['inteiroTeor'].isnull()),
                  ['statusExtracao','tipoDocumento']] = [5,'imagem salva como pdf']

In [125]:
dfInteiroTeor.to_csv(f'{DIR_DATASET}inteiroTeor.csv',index=False,sep=';',quoting=csv.QUOTE_ALL,  escapechar="\\")
print(f'[OK    ] Arquivo salvo: {DIR_DATASET}inteiroTeor.csv')

[OK    ] Arquivo salvo: /Users/dev-rocks/Documents/TCC - Data Science e Big Data/Projeto/datasets/inteiroTeor.csv
